In [2]:
!pip install opencv-python
# !pip install cvlib
# !pip install tensorflow

In [1]:
import cv2

In [4]:
sunglasses = cv2.imread('sunglasses-eyewear-clip-art-alpha-channel-70ca12645c25ddaa2688f2fb32b35622.png', cv2.IMREAD_UNCHANGED)
if sunglasses is None:
    print("Error: 'sunglasses-eyewear-clip-art-alpha-channel-70ca12645c25ddaa2688f2fb32b35622.png' not found.")

img = cv2.imread("emma_watson.jpg")
if img is not None:
    img = cv2.resize(img, (500, 500)) 
if img is None:
    print("Error: Image 'emma_watson.jpg' not found or cannot be loaded.")
else:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        print("No face detected.")
    else:
        for (fx, fy, fw, fh) in faces:
            roi_gray = gray[fy:fy+fh, fx:fx+fw]
            roi_color = img[fy:fy+fh, fx:fx+fw]
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
            if sunglasses.shape[2] < 4:
                print("Error: 'sunglasses.png' does not have an alpha channel.")
            elif len(eyes) >= 2:
                # Sort eyes by x position
                eyes = sorted(eyes, key=lambda x: x[0])
                ex1, ey1, ew1, eh1 = eyes[0]
                ex2, ey2, ew2, eh2 = eyes[1]
                # Calculate center between eyes
                eye_center_x = int((ex1 + ex2 + ew1 // 2 + ew2 // 2) / 2)
                eye_center_y = int((ey1 + ey2 + eh1 // 2 + eh2 // 2) / 2)
                # Calculate sunglasses width as distance between eyes plus some padding
                sg_width = int(abs((ex2 + ew2 // 2) - (ex1 + ew1 // 2)) * 2)
                sg_height = int(sg_width * sunglasses.shape[0] / sunglasses.shape[1])
                # Top-left corner for overlay
                x1 = eye_center_x - sg_width // 2
                y1 = eye_center_y - sg_height // 2
                # Ensure within ROI bounds
                x1 = max(x1, 0)
                y1 = max(y1, 0)
                x2 = min(x1 + sg_width, roi_color.shape[1])
                y2 = min(y1 + sg_height, roi_color.shape[0])
                # Resize sunglasses
                sg_resized = cv2.resize(sunglasses, (x2 - x1, y2 - y1))
                sg_rgb = sg_resized[:, :, :3]
                sg_alpha = sg_resized[:, :, 3] / 255.0
                # Overlay sunglasses
                for c in range(3):
                    roi_color[y1:y2, x1:x2, c] = (
                        sg_rgb[:, :, c] * sg_alpha +
                        roi_color[y1:y2, x1:x2, c] * (1 - sg_alpha)
                    ).astype('uint8')
        cv2.namedWindow("Sunglasses Overlay", cv2.WINDOW_NORMAL)
        cv2.imshow("Sunglasses Overlay", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

## Robust Detection

In [20]:
import cv2
import numpy as np

def overlay_sunglasses(img_path, sunglasses_path):
    sunglasses = cv2.imread(sunglasses_path, cv2.IMREAD_UNCHANGED)
    if sunglasses is None or sunglasses.shape[2] < 4:
        print("Error: Sunglasses image missing or no alpha channel.")
        return

    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Image '{img_path}' not found or cannot be loaded.")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        print("No face detected. Try adjusting scaleFactor or minNeighbors.")
        return

    for (fx, fy, fw, fh) in faces:
        roi_gray = gray[fy:fy+fh, fx:fx+fw]
        roi_color = img[fy:fy+fh, fx:fx+fw]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
        # Debug: Draw rectangles
        # for (ex, ey, ew, eh) in eyes:
        #     cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0,255,0), 2)

        if len(eyes) < 2:
            print("Less than two eyes detected. Try adjusting parameters or using a clearer image.")
            continue

        # Use two eyes with largest width (likely actual eyes)
        eyes = sorted(eyes, key=lambda x: -x[2])[:2]
        eyes = sorted(eyes, key=lambda x: x[0])  # left-right order

        ex1, ey1, ew1, eh1 = eyes[0]
        ex2, ey2, ew2, eh2 = eyes[1]
        eye_center_x = int((ex1 + ex2 + ew1 // 2 + ew2 // 2) / 2)
        eye_center_y = int((ey1 + ey2 + eh1 // 2 + eh2 // 2) / 2)
        sg_width = int(abs((ex2 + ew2 // 2) - (ex1 + ew1 // 2)) * 2.2)
        sg_height = int(sg_width * sunglasses.shape[0] / sunglasses.shape[1])

        x1 = max(eye_center_x - sg_width // 2, 0)
        y1 = max(eye_center_y - sg_height // 2, 0)
        x2 = min(x1 + sg_width, roi_color.shape[1])
        y2 = min(y1 + sg_height, roi_color.shape[0])

        sg_resized = cv2.resize(sunglasses, (x2 - x1, y2 - y1))
        sg_rgb = sg_resized[:, :, :3]
        sg_alpha = sg_resized[:, :, 3] / 255.0

        for c in range(3):
            roi_color[y1:y2, x1:x2, c] = (
                sg_rgb[:, :, c] * sg_alpha +
                roi_color[y1:y2, x1:x2, c] * (1 - sg_alpha)
            ).astype('uint8')

    cv2.imshow("Sunglasses Overlay", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Usage
overlay_sunglasses("The Rock.jpeg", "sunglasses-eyewear-clip-art-alpha-channel-70ca12645c25ddaa2688f2fb32b35622.png")